In [1]:
%%capture
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,ArrayType
from pyspark.sql.functions import col,lit,array_contains
from pyspark.sql.functions import sum,avg,max,min,mean,count
from pyspark.sql.functions import udf

In [3]:
spark = SparkSession.builder.master('local[*]').appName("SparkTest").getOrCreate()

### Create DataFrame

In [4]:
data = [("James","","Smith","36636","M",3000),
    ("Michael","Rose","","40288","M",4000),
    ("Robert","","Williams","42114","M",4000),
    ("Maria","Anne","Jones","39192","F",4000),
    ("Jen","Mary","Brown","","F",-1)
  ]

schema = StructType([ \
    StructField("firstname",StringType(),True), \
    StructField("middlename",StringType(),True), \
    StructField("lastname",StringType(),True), \
    StructField("id", StringType(), True), \
    StructField("gender", StringType(), True), \
    StructField("salary", IntegerType(), True) \
  ])
 
df = spark.createDataFrame(data=data,schema=schema)
df.printSchema()
df.show(truncate=False)

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)

+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|id   |gender|salary|
+---------+----------+--------+-----+------+------+
|James    |          |Smith   |36636|M     |3000  |
|Michael  |Rose      |        |40288|M     |4000  |
|Robert   |          |Williams|42114|M     |4000  |
|Maria    |Anne      |Jones   |39192|F     |4000  |
|Jen      |Mary      |Brown   |     |F     |-1    |
+---------+----------+--------+-----+------+------+



In [5]:
pandasDF = df.toPandas()
print(pandasDF)

  firstname middlename  lastname     id gender  salary
0     James                Smith  36636      M    3000
1   Michael       Rose            40288      M    4000
2    Robert             Williams  42114      M    4000
3     Maria       Anne     Jones  39192      F    4000
4       Jen       Mary     Brown             F      -1


### Select columns from PySpark

In [6]:
df.select(col("firstname"),col("lastname")).show()

+---------+--------+
|firstname|lastname|
+---------+--------+
|    James|   Smith|
|  Michael|        |
|   Robert|Williams|
|    Maria|   Jones|
|      Jen|   Brown|
+---------+--------+



In [7]:
df.select(df.colRegex("`^.*name*`")).show()

+---------+----------+--------+
|firstname|middlename|lastname|
+---------+----------+--------+
|    James|          |   Smith|
|  Michael|      Rose|        |
|   Robert|          |Williams|
|    Maria|      Anne|   Jones|
|      Jen|      Mary|   Brown|
+---------+----------+--------+



### Create new column

In [8]:
df = df.withColumn("percent",col("salary")/100*10)
df.show()

+---------+----------+--------+-----+------+------+-------+
|firstname|middlename|lastname|   id|gender|salary|percent|
+---------+----------+--------+-----+------+------+-------+
|    James|          |   Smith|36636|     M|  3000|  300.0|
|  Michael|      Rose|        |40288|     M|  4000|  400.0|
|   Robert|          |Williams|42114|     M|  4000|  400.0|
|    Maria|      Anne|   Jones|39192|     F|  4000|  400.0|
|      Jen|      Mary|   Brown|     |     F|    -1|   -0.1|
+---------+----------+--------+-----+------+------+-------+



### Add new column

In [9]:
df = df.withColumn("Country", lit("USA"))
df.show()

+---------+----------+--------+-----+------+------+-------+-------+
|firstname|middlename|lastname|   id|gender|salary|percent|Country|
+---------+----------+--------+-----+------+------+-------+-------+
|    James|          |   Smith|36636|     M|  3000|  300.0|    USA|
|  Michael|      Rose|        |40288|     M|  4000|  400.0|    USA|
|   Robert|          |Williams|42114|     M|  4000|  400.0|    USA|
|    Maria|      Anne|   Jones|39192|     F|  4000|  400.0|    USA|
|      Jen|      Mary|   Brown|     |     F|    -1|   -0.1|    USA|
+---------+----------+--------+-----+------+------+-------+-------+



### Rename column name

In [10]:
df = df.withColumnRenamed("Country","country")
df.show(truncate=False) 

+---------+----------+--------+-----+------+------+-------+-------+
|firstname|middlename|lastname|id   |gender|salary|percent|country|
+---------+----------+--------+-----+------+------+-------+-------+
|James    |          |Smith   |36636|M     |3000  |300.0  |USA    |
|Michael  |Rose      |        |40288|M     |4000  |400.0  |USA    |
|Robert   |          |Williams|42114|M     |4000  |400.0  |USA    |
|Maria    |Anne      |Jones   |39192|F     |4000  |400.0  |USA    |
|Jen      |Mary      |Brown   |     |F     |-1    |-0.1   |USA    |
+---------+----------+--------+-----+------+------+-------+-------+



### Drop column from dataframe

In [11]:
df = df.drop("country")
df.show()

+---------+----------+--------+-----+------+------+-------+
|firstname|middlename|lastname|   id|gender|salary|percent|
+---------+----------+--------+-----+------+------+-------+
|    James|          |   Smith|36636|     M|  3000|  300.0|
|  Michael|      Rose|        |40288|     M|  4000|  400.0|
|   Robert|          |Williams|42114|     M|  4000|  400.0|
|    Maria|      Anne|   Jones|39192|     F|  4000|  400.0|
|      Jen|      Mary|   Brown|     |     F|    -1|   -0.1|
+---------+----------+--------+-----+------+------+-------+



### Dataframe filter

In [12]:
data2 = [
    (("James","","Smith"),["Java","Scala","C++"],"OH","M"),
    (("Anna","Rose",""),["Spark","Java","C++"],"NY","F"),
    (("Julia","","Williams"),["CSharp","VB"],"OH","F"),
    (("Maria","Anne","Jones"),["CSharp","VB"],"NY","M"),
    (("Jen","Mary","Brown"),["CSharp","VB"],"NY","M"),
    (("Mike","Mary","Williams"),["Python","VB"],"OH","M")
 ]
        
schema2 = StructType([
     StructField('name', StructType([
        StructField('firstname', StringType(), True),
        StructField('middlename', StringType(), True),
         StructField('lastname', StringType(), True)
     ])),
     StructField('languages', ArrayType(StringType()), True),
     StructField('state', StringType(), True),
     StructField('gender', StringType(), True)
 ])

df2 = spark.createDataFrame(data = data2, schema = schema2)
df2.printSchema()
df2.show(truncate=False)

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- languages: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- state: string (nullable = true)
 |-- gender: string (nullable = true)

+----------------------+------------------+-----+------+
|name                  |languages         |state|gender|
+----------------------+------------------+-----+------+
|{James, , Smith}      |[Java, Scala, C++]|OH   |M     |
|{Anna, Rose, }        |[Spark, Java, C++]|NY   |F     |
|{Julia, , Williams}   |[CSharp, VB]      |OH   |F     |
|{Maria, Anne, Jones}  |[CSharp, VB]      |NY   |M     |
|{Jen, Mary, Brown}    |[CSharp, VB]      |NY   |M     |
|{Mike, Mary, Williams}|[Python, VB]      |OH   |M     |
+----------------------+------------------+-----+------+



In [13]:
df2.filter(df2.state == "OH").show(truncate=False)

+----------------------+------------------+-----+------+
|name                  |languages         |state|gender|
+----------------------+------------------+-----+------+
|{James, , Smith}      |[Java, Scala, C++]|OH   |M     |
|{Julia, , Williams}   |[CSharp, VB]      |OH   |F     |
|{Mike, Mary, Williams}|[Python, VB]      |OH   |M     |
+----------------------+------------------+-----+------+



In [14]:
df2.filter("gender == 'M'").show()

+--------------------+------------------+-----+------+
|                name|         languages|state|gender|
+--------------------+------------------+-----+------+
|    {James, , Smith}|[Java, Scala, C++]|   OH|     M|
|{Maria, Anne, Jones}|      [CSharp, VB]|   NY|     M|
|  {Jen, Mary, Brown}|      [CSharp, VB]|   NY|     M|
|{Mike, Mary, Will...|      [Python, VB]|   OH|     M|
+--------------------+------------------+-----+------+



In [15]:
li=["OH","CA","DE"]
df2.filter(df2.state.isin(li)).show()

+--------------------+------------------+-----+------+
|                name|         languages|state|gender|
+--------------------+------------------+-----+------+
|    {James, , Smith}|[Java, Scala, C++]|   OH|     M|
| {Julia, , Williams}|      [CSharp, VB]|   OH|     F|
|{Mike, Mary, Will...|      [Python, VB]|   OH|     M|
+--------------------+------------------+-----+------+



In [16]:
df2.filter(array_contains(df2.languages,"Java")).show(truncate=False)

+----------------+------------------+-----+------+
|name            |languages         |state|gender|
+----------------+------------------+-----+------+
|{James, , Smith}|[Java, Scala, C++]|OH   |M     |
|{Anna, Rose, }  |[Spark, Java, C++]|NY   |F     |
+----------------+------------------+-----+------+



### Distinct / Drop Duplicate Rows

In [17]:
data3 = [("James", "Sales", 3000), \
    ("Michael", "Sales", 4600), \
    ("Robert", "Sales", 4100), \
    ("Maria", "Finance", 3000), \
    ("James", "Sales", 3000), \
    ("Scott", "Finance", 3300), \
    ("Jen", "Finance", 3900), \
    ("Jeff", "Marketing", 3000), \
    ("Kumar", "Marketing", 2000), \
    ("Saif", "Sales", 4100) \
  ]
columns3= ["employee_name", "department", "salary"]
df3 = spark.createDataFrame(data = data3, schema = columns3)
df3.printSchema()
df3.show(truncate=False)

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|James        |Sales     |3000  |
|Michael      |Sales     |4600  |
|Robert       |Sales     |4100  |
|Maria        |Finance   |3000  |
|James        |Sales     |3000  |
|Scott        |Finance   |3300  |
|Jen          |Finance   |3900  |
|Jeff         |Marketing |3000  |
|Kumar        |Marketing |2000  |
|Saif         |Sales     |4100  |
+-------------+----------+------+



In [18]:
distinctDF = df3.distinct()
print("Distinct count: "+str(distinctDF.count()))
distinctDF.show(truncate=False)

Distinct count: 9
+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|Michael      |Sales     |4600  |
|James        |Sales     |3000  |
|Robert       |Sales     |4100  |
|Maria        |Finance   |3000  |
|Jen          |Finance   |3900  |
|Scott        |Finance   |3300  |
|Kumar        |Marketing |2000  |
|Jeff         |Marketing |3000  |
|Saif         |Sales     |4100  |
+-------------+----------+------+



In [19]:
distinctDF_ = df3.dropDuplicates()
print("Distinct count: "+str(distinctDF_.count()))
distinctDF_.show(truncate=False)

Distinct count: 9
+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|Michael      |Sales     |4600  |
|James        |Sales     |3000  |
|Robert       |Sales     |4100  |
|Maria        |Finance   |3000  |
|Jen          |Finance   |3900  |
|Scott        |Finance   |3300  |
|Kumar        |Marketing |2000  |
|Jeff         |Marketing |3000  |
|Saif         |Sales     |4100  |
+-------------+----------+------+



### Dataframe orderBy and sort

In [20]:
data4 = [("James","Sales","NY",90000,34,10000), \
    ("Michael","Sales","NY",86000,56,20000), \
    ("Robert","Sales","CA",81000,30,23000), \
    ("Maria","Finance","CA",90000,24,23000), \
    ("Raman","Finance","CA",99000,40,24000), \
    ("Scott","Finance","NY",83000,36,19000), \
    ("Jen","Finance","NY",79000,53,15000), \
    ("Jeff","Marketing","CA",80000,25,18000), \
    ("Kumar","Marketing","NY",91000,50,21000) \
  ]
columns4= ["employee_name","department","state","salary","age","bonus"]
df4 = spark.createDataFrame(data = data4, schema = columns4)
df4.printSchema()
df4.show(truncate=False)

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- state: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- age: long (nullable = true)
 |-- bonus: long (nullable = true)

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|James        |Sales     |NY   |90000 |34 |10000|
|Michael      |Sales     |NY   |86000 |56 |20000|
|Robert       |Sales     |CA   |81000 |30 |23000|
|Maria        |Finance   |CA   |90000 |24 |23000|
|Raman        |Finance   |CA   |99000 |40 |24000|
|Scott        |Finance   |NY   |83000 |36 |19000|
|Jen          |Finance   |NY   |79000 |53 |15000|
|Jeff         |Marketing |CA   |80000 |25 |18000|
|Kumar        |Marketing |NY   |91000 |50 |21000|
+-------------+----------+-----+------+---+-----+



In [21]:
df4.sort(col("department").asc(),col("state").desc()).show(truncate=False)

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|Scott        |Finance   |NY   |83000 |36 |19000|
|Jen          |Finance   |NY   |79000 |53 |15000|
|Raman        |Finance   |CA   |99000 |40 |24000|
|Maria        |Finance   |CA   |90000 |24 |23000|
|Kumar        |Marketing |NY   |91000 |50 |21000|
|Jeff         |Marketing |CA   |80000 |25 |18000|
|Michael      |Sales     |NY   |86000 |56 |20000|
|James        |Sales     |NY   |90000 |34 |10000|
|Robert       |Sales     |CA   |81000 |30 |23000|
+-------------+----------+-----+------+---+-----+



In [22]:
df4.orderBy(col("department").asc(),col("state").asc()).show(truncate=False)

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|Raman        |Finance   |CA   |99000 |40 |24000|
|Maria        |Finance   |CA   |90000 |24 |23000|
|Scott        |Finance   |NY   |83000 |36 |19000|
|Jen          |Finance   |NY   |79000 |53 |15000|
|Jeff         |Marketing |CA   |80000 |25 |18000|
|Kumar        |Marketing |NY   |91000 |50 |21000|
|Robert       |Sales     |CA   |81000 |30 |23000|
|James        |Sales     |NY   |90000 |34 |10000|
|Michael      |Sales     |NY   |86000 |56 |20000|
+-------------+----------+-----+------+---+-----+



### Dataframe groupby 

In [23]:
data5 = [("James","Sales","NY",90000,34,10000),
    ("Michael","Sales","NY",86000,56,20000),
    ("Robert","Sales","CA",81000,30,23000),
    ("Maria","Finance","CA",90000,24,23000),
    ("Raman","Finance","CA",99000,40,24000),
    ("Scott","Finance","NY",83000,36,19000),
    ("Jen","Finance","NY",79000,53,15000),
    ("Jeff","Marketing","CA",80000,25,18000),
    ("Kumar","Marketing","NY",91000,50,21000)
  ]

columns5 = ["employee_name","department","state","salary","age","bonus"]
df5 = spark.createDataFrame(data=data5, schema = columns5)
df5.printSchema()
df5.show(truncate=False)

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- state: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- age: long (nullable = true)
 |-- bonus: long (nullable = true)

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|James        |Sales     |NY   |90000 |34 |10000|
|Michael      |Sales     |NY   |86000 |56 |20000|
|Robert       |Sales     |CA   |81000 |30 |23000|
|Maria        |Finance   |CA   |90000 |24 |23000|
|Raman        |Finance   |CA   |99000 |40 |24000|
|Scott        |Finance   |NY   |83000 |36 |19000|
|Jen          |Finance   |NY   |79000 |53 |15000|
|Jeff         |Marketing |CA   |80000 |25 |18000|
|Kumar        |Marketing |NY   |91000 |50 |21000|
+-------------+----------+-----+------+---+-----+



In [24]:
#count(), mean(), max(), min(), sum(), avg()
df5.groupBy("department").sum("salary").show(truncate=False)

+----------+-----------+
|department|sum(salary)|
+----------+-----------+
|Sales     |257000     |
|Finance   |351000     |
|Marketing |171000     |
+----------+-----------+



In [25]:
df5.groupBy("department","state").sum("salary","bonus").show(truncate=False)

+----------+-----+-----------+----------+
|department|state|sum(salary)|sum(bonus)|
+----------+-----+-----------+----------+
|Sales     |CA   |81000      |23000     |
|Finance   |CA   |189000     |47000     |
|Sales     |NY   |176000     |30000     |
|Finance   |NY   |162000     |34000     |
|Marketing |NY   |91000      |21000     |
|Marketing |CA   |80000      |18000     |
+----------+-----+-----------+----------+



In [26]:
df5.groupBy("department"). \
                          agg(sum("salary").alias("sum_salary"), \
                              avg("salary").alias("avg_salary"), \
                              sum("bonus").alias("sum_bonus"), \
                              max("bonus").alias("max_bonus")). \
                          show(truncate=False)

+----------+----------+-----------------+---------+---------+
|department|sum_salary|avg_salary       |sum_bonus|max_bonus|
+----------+----------+-----------------+---------+---------+
|Sales     |257000    |85666.66666666667|53000    |23000    |
|Finance   |351000    |87750.0          |81000    |24000    |
|Marketing |171000    |85500.0          |39000    |21000    |
+----------+----------+-----------------+---------+---------+



### PySpark Join types

In [27]:
data6 = [(1,"Smith",-1,"2018","10","M",3000), \
    (2,"Rose",1,"2010","20","M",4000), \
    (3,"Williams",1,"2010","10","M",1000), \
    (4,"Jones",2,"2005","10","F",2000), \
    (5,"Brown",2,"2010","40","",-1), \
      (6,"Brown",2,"2010","50","",-1) \
  ]
columns6 = ["emp_id","name","superior_emp_id","year_joined", \
       "emp_dept_id","gender","salary"]

df6 = spark.createDataFrame(data=data6, schema = columns6)
df6.printSchema()
df6.show(truncate=False)

data7 = [("Finance",10), \
    ("Marketing",20), \
    ("Sales",30), \
    ("IT",40) \
  ]
columns7 = ["dept_name","dept_id"]
df7 = spark.createDataFrame(data=data7, schema = columns7)
df7.printSchema()
df7.show(truncate=False)

root
 |-- emp_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- superior_emp_id: long (nullable = true)
 |-- year_joined: string (nullable = true)
 |-- emp_dept_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |
|2     |Rose    |1              |2010       |20         |M     |4000  |
|3     |Williams|1              |2010       |10         |M     |1000  |
|4     |Jones   |2              |2005       |10         |F     |2000  |
|5     |Brown   |2              |2010       |40         |      |-1    |
|6     |Brown   |2              |2010       |50         |      |-1    |
+------+--------+---------------+-----------+-----------+------+-----

In [28]:
df6.join(df7,df6.emp_dept_id ==  df7.dept_id,"left").show(truncate=False)

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
|6     |Brown   |2              |2010       |50         |      |-1    |null     |null   |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



### Dataframe union and unionByName (if two dataframes with different columns or schema)

In [29]:
data8 = [("James","Sales","NY",90000,34,10000), \
    ("Maria","Finance","CA",90000,24,23000) \
  ]

columns8= ["employee_name","department","state","salary","age","bonus"]
df8 = spark.createDataFrame(data = data8, schema = columns8)
df8.printSchema()
df8.show(truncate=False)


data9 = [("James","Sales","NY",90000,34,10000), \
    ("Robert","Sales","CA",81000,30,23000), \
    ("Maria","Finance","CA",90000,24,23000) \
  ]

columns9= ["employee_name","department","state","salary","age","bonus"]
df9 = spark.createDataFrame(data = data9, schema = columns9)
df9.printSchema()
df9.show(truncate=False)


root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- state: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- age: long (nullable = true)
 |-- bonus: long (nullable = true)

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|James        |Sales     |NY   |90000 |34 |10000|
|Maria        |Finance   |CA   |90000 |24 |23000|
+-------------+----------+-----+------+---+-----+

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- state: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- age: long (nullable = true)
 |-- bonus: long (nullable = true)

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|James        |Sales     |NY   |90000 |34 |10000|
|Robert       |Sales     |CA   

In [30]:
df10 = df8.union(df9).distinct()
df10.show(truncate=False)

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|James        |Sales     |NY   |90000 |34 |10000|
|Maria        |Finance   |CA   |90000 |24 |23000|
|Robert       |Sales     |CA   |81000 |30 |23000|
+-------------+----------+-----+------+---+-----+



### PySpark UDF

In [31]:
data11 = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders")]

columns11 = ["User_id","Name"]

df11 = spark.createDataFrame(data=data11,schema=columns11)
df11.show(truncate=False)

+-------+------------+
|User_id|Name        |
+-------+------------+
|1      |john jones  |
|2      |tracey smith|
|3      |amy sanders |
+-------+------------+



In [32]:
def convertCase(str):
    resStr=""
    arr = str.split(" ")
    for x in arr:
       resStr= resStr + x[0:1].upper() + x[1:len(x)] + " "
    return resStr 

In [33]:
convertUDF = udf(lambda z: convertCase(z),StringType())

In [34]:
df11.select(col("User_id"),convertUDF(col("Name")).alias("Name")).show(truncate=False)

+-------+-------------+
|User_id|Name         |
+-------+-------------+
|1      |John Jones   |
|2      |Tracey Smith |
|3      |Amy Sanders  |
+-------+-------------+



### Pivot dataframe

In [35]:
data12 = [("Banana",1000,"USA"), ("Carrots",1500,"USA"), ("Beans",1600,"USA"), \
      ("Orange",2000,"USA"),("Orange",2000,"USA"),("Banana",400,"China"), \
      ("Carrots",1200,"China"),("Beans",1500,"China"),("Orange",4000,"China"), \
      ("Banana",2000,"Canada"),("Carrots",2000,"Canada"),("Beans",2000,"Mexico")]

columns12 = ["Product","Amount","Country"]
df12 = spark.createDataFrame(data = data12, schema = columns12)
df12.printSchema()
df12.show(truncate=False)

root
 |-- Product: string (nullable = true)
 |-- Amount: long (nullable = true)
 |-- Country: string (nullable = true)

+-------+------+-------+
|Product|Amount|Country|
+-------+------+-------+
|Banana |1000  |USA    |
|Carrots|1500  |USA    |
|Beans  |1600  |USA    |
|Orange |2000  |USA    |
|Orange |2000  |USA    |
|Banana |400   |China  |
|Carrots|1200  |China  |
|Beans  |1500  |China  |
|Orange |4000  |China  |
|Banana |2000  |Canada |
|Carrots|2000  |Canada |
|Beans  |2000  |Mexico |
+-------+------+-------+



In [36]:
countries = ["USA","China","Canada"]
pivot_df12 = df12.groupBy("Product").pivot("Country",countries).sum("Amount")
pivot_df12.printSchema()
pivot_df12.show(truncate=False)

root
 |-- Product: string (nullable = true)
 |-- USA: long (nullable = true)
 |-- China: long (nullable = true)
 |-- Canada: long (nullable = true)

+-------+----+-----+------+
|Product|USA |China|Canada|
+-------+----+-----+------+
|Orange |4000|4000 |null  |
|Beans  |1600|1500 |null  |
|Banana |1000|400  |2000  |
|Carrots|1500|1200 |2000  |
+-------+----+-----+------+



In [37]:
pivot_df12.na.fill(value=0).show()

+-------+----+-----+------+
|Product| USA|China|Canada|
+-------+----+-----+------+
| Orange|4000| 4000|     0|
|  Beans|1600| 1500|     0|
| Banana|1000|  400|  2000|
|Carrots|1500| 1200|  2000|
+-------+----+-----+------+



### PySpark partitionBy

In [38]:
# Удаление созданного каталога 
# !rm -Rf /content/tmp

In [39]:
df12.write.option("header",True) \
        .partitionBy("Product") \
        .mode("overwrite") \
        .csv("/content/tmp/sales_product")

In [40]:
df12_=spark.read.option("header",True).csv("/content/tmp/sales_product")
df12_.printSchema()
df12_.show()

root
 |-- Amount: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Product: string (nullable = true)

+------+-------+-------+
|Amount|Country|Product|
+------+-------+-------+
|  1200|  China|Carrots|
|  2000| Canada|Carrots|
|  1500|  China|  Beans|
|  2000| Mexico|  Beans|
|  1000|    USA| Banana|
|   400|  China| Banana|
|  2000|    USA| Orange|
|  2000|    USA| Orange|
|  2000| Canada| Banana|
|  4000|  China| Orange|
|  1500|    USA|Carrots|
|  1600|    USA|  Beans|
+------+-------+-------+



### Read and write parquet file

In [41]:
df12.write.option("header",True) \
            .partitionBy("Product") \
            .mode('overwrite') \
            .parquet("/content/tmp/parquet/sales.parquet")

In [42]:
df12_par=spark.read.parquet("/content/tmp/parquet/sales.parquet")
df12_par.show(truncate=False)

+------+-------+-------+
|Amount|Country|Product|
+------+-------+-------+
|2000  |USA    |Orange |
|2000  |USA    |Orange |
|2000  |Canada |Banana |
|1200  |China  |Carrots|
|2000  |Canada |Carrots|
|1500  |China  |Beans  |
|2000  |Mexico |Beans  |
|4000  |China  |Orange |
|1000  |USA    |Banana |
|400   |China  |Banana |
|1500  |USA    |Carrots|
|1600  |USA    |Beans  |
+------+-------+-------+



In [44]:
spark.sql("DROP VIEW IF EXISTS SALES")
spark.sql("CREATE TEMPORARY VIEW SALES USING parquet OPTIONS (path \"/content/tmp/parquet/sales.parquet\")")
spark.sql("SELECT * FROM SALES" ).show()

+------+-------+-------+
|Amount|Country|Product|
+------+-------+-------+
|  2000|    USA| Orange|
|  2000|    USA| Orange|
|  2000| Canada| Banana|
|  1200|  China|Carrots|
|  2000| Canada|Carrots|
|  1500|  China|  Beans|
|  2000| Mexico|  Beans|
|  4000|  China| Orange|
|  1000|    USA| Banana|
|   400|  China| Banana|
|  1500|    USA|Carrots|
|  1600|    USA|  Beans|
+------+-------+-------+

